In [1]:
import numpy as np 
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter

In [2]:
def select_train(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

def select_test(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

In [4]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()

# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset1 = raw_dataset.copy()
test_dataset1.isna().sum()
test_dataset1 = test_dataset1.dropna()

# 2/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim2.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset2 = raw_dataset.copy()
test_dataset2.isna().sum()
test_dataset2 = test_dataset2.dropna()

# 3/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim3.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset3 = raw_dataset.copy()
test_dataset3.isna().sum()
test_dataset3 = test_dataset3.dropna()

In [5]:
list_comp = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']

list_var = ['input_width']

train_dataset = select_train(list_var, train_dataset)
test_dataset1 = select_test(list_var, test_dataset1)
test_dataset2 = select_test(list_var, test_dataset2)
test_dataset3 = select_test(list_var, test_dataset3)

In [6]:
y_train = train_dataset.pop('memory')
X_train = train_dataset
X_train.tail()

,Model_name,input_shape,input_size,nb_layers,sum_activations,params
149,basic_model_350_45,"(1, 350, 350, 3)",367500,77,24032836,28087872
150,basic_model_224_24,"(1, 224, 224, 3)",150528,44,18477760,49420384
151,basic_model_240_26,"(1, 240, 240, 3)",172800,44,335053672,36989976
152,basic_model_512_28,"(1, 512, 512, 3)",786432,54,383584308,34196142
153,basic_model_224_33,"(1, 224, 224, 3)",150528,60,31316236,22456930


In [7]:
Y_e1 = test_dataset1.pop('memory')
X_e1 = test_dataset1

Y_e2 = test_dataset2.pop('memory')
X_e2 = test_dataset2

Y_e3 = test_dataset3.pop('memory')
X_e3 = test_dataset3

In [8]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

X_test_model_names1 = X_e1.pop('Model_name')
X_test_input_shapes1 = X_e1.pop('input_shape')

X_test_model_names2 = X_e2.pop('Model_name')
X_test_input_shapes2 = X_e2.pop('input_shape')

X_test_model_names3 = X_e3.pop('Model_name')
X_test_input_shapes3 = X_e3.pop('input_shape')

In [9]:
y_test1 = Y_e1.values
y_test2 = Y_e2.values
y_test3 = Y_e3.values

In [10]:
steps = [('scaler', StandardScaler()), ('SVR', SVR())]
pipeline = Pipeline(steps)

In [11]:
# Hyperparameters space
"""
parameteres = {'SVR__kernel':['linear', 'rbf', 'poly'], 
'SVR__cache_size':[5000000], 
'SVR__C':[1000, 5000, 8000, 10000, 50000, 100000, 200000], 
'SVR__gamma':[.1, .3, .5, .7, .8, .9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
'SVR__epsilon':[.1, .3, .5, .7, .9, 1., 1.5, 2., 2.5, 3., 3.5, 4, 4.5, 5]}
"""

# Optimal Hyperparameters 
# rbf 10000
parameteres = {'SVR__kernel':['rbf'], 'SVR__cache_size':[5000000], 'SVR__C':[1000], 'SVR__gamma':[.6]}

In [12]:
grid = GridSearchCV(pipeline, param_grid=parameteres, cv=2)

In [13]:
grid.fit(X_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('SVR', SVR())]),
             param_grid={'SVR__C': [1000], 'SVR__cache_size': [5000000],
                         'SVR__gamma': [0.6], 'SVR__kernel': ['rbf']})

In [14]:
print(grid.best_params_)

{'SVR__C': 1000, 'SVR__cache_size': 5000000, 'SVR__gamma': 0.6, 'SVR__kernel': 'rbf'}


In [15]:
best_pipe = grid.best_estimator_
best_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('SVR', SVR(C=1000, cache_size=5000000, gamma=0.6))])

In [16]:
#%%timeit -n 1000
#predictions = best_pipe.predict(np.log(X_test))

In [17]:
y_pred1 = best_pipe.predict(X_e1.values)
y_pred2 = best_pipe.predict(X_e2.values)
y_pred3 = best_pipe.predict(X_e3.values)

In [26]:
y_test = y_test3
y_pred = y_pred3
test_models = X_test_model_names3.values
test_input_shapes = X_test_input_shapes3.values

In [27]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.8473173507065995
R2_value :  0.8460748275126615
Mean absolute error MAE : 211.89427481812723
Root Mean squared error RMSE : 329.5330079671528
Median absolute error MAD: 121.70514250133874


In [28]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE :', max(mape))
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE : 42.8190320927411
MAPE: 8.57 %.
Accuracy: 91.43 %.


In [29]:
import csv
with open('./measured_vs_predicted_SVR_dim3.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])